In [1]:
import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir)

In [2]:
import pandas as pd
from bayesian.train_bn import structure_learning, parameter_learning
from preprocess.discretization import get_nodes_type, discretization, code_categories

from bayesian.save_bn import save_structure, save_params, read_structure, read_params
from external.libpgm.hybayesiannetwork import HyBayesianNetwork
from bayesian.calculate_accuracy import calculate_acc, LOO_validation

from copy import copy
from scipy import stats

import random
from sklearn.model_selection import train_test_split

In [3]:
random.seed(2000)

In [4]:
geo = pd.read_csv('../data/hackathon_processed.csv')
columns = ['Tectonic regime', 'Period', 'Lithology', 'Structural setting', 'Hydrocarbon type', 'Gross','Netpay','Porosity','Permeability', 'Depth']
geo = geo[columns]
geo.dropna(inplace=True)
geo.reset_index(inplace=True, drop=True)
#geo = geo[:100]

In [5]:
df_result = pd.DataFrame(columns = ['Method', 'isDisc', 'isSimple', 'isLogit'].extend(columns))
path = 'geo_noLoo_results_logit.csv'
method_list = ['K2', 'MI', 'LL', 'AIC', 'BIC']
#method_list = ['AIC']

In [6]:
geo_train, geo_test = train_test_split(geo, test_size=0.1)
geo_train.reset_index(inplace=True, drop=True)
geo_test.reset_index(inplace=True, drop=True)

In [7]:
geo_types = get_nodes_type(geo)
geo_types

{'Tectonic regime': 'disc',
 'Period': 'disc',
 'Lithology': 'disc',
 'Structural setting': 'disc',
 'Hydrocarbon type': 'disc',
 'Gross': 'cont',
 'Netpay': 'cont',
 'Porosity': 'cont',
 'Permeability': 'cont',
 'Depth': 'cont'}

In [8]:
colums_for_code = []
columns_for_disc = []
for c in columns:
    if geo_types[c] == 'disc':
        colums_for_code.append(c)
    else:
        columns_for_disc.append(c)

In [9]:
geo_coded, label_coder = code_categories(geo_train, 'label', colums_for_code)
geo_discrete, coder = discretization(geo_coded, 'kmeans', columns_for_disc)
geo_only_discrete, discrete_coder = discretization(geo_train, 'kmeans', columns_for_disc)

# Logit simple

In [10]:
for method in method_list:
    bn_geo = structure_learning(geo_discrete, 'HC', geo_types, method, cont_disc = True)
    params = parameter_learning(geo_train, geo_types, bn_geo, 'simple')
    save_structure(bn_geo, path)
    skel = read_structure(path)
    save_params(params, f'{path}_param')
    params = read_params(f'{path}_param')
    geo_bn = HyBayesianNetwork(skel, params)

    acc, rmse, real_param, pred_param, indexes = calculate_acc(geo_bn, geo_test, columns, 'simple', normed = True)
    df_result = df_result.append({'Method': method, 'isDisc': 'D', 'isSimple': 'S', 'isLogit': 'L', **acc, **rmse}, ignore_index=True)
    df_result.to_csv(path, index=False)
    df_result = pd.read_csv(path)

    if method != 'K2':
        bn_geo = structure_learning(geo_coded, 'HC', geo_types, method, cont_disc = True)
        params = parameter_learning(geo_train, geo_types, bn_geo, 'simple')
        save_structure(bn_geo, path)
        skel = read_structure(path)
        save_params(params, f'{path}_param')
        params = read_params(f'{path}_param')
        geo_bn = HyBayesianNetwork(skel, params)

        acc, rmse, real_param, pred_param, indexes = calculate_acc(geo_bn, geo_test, columns, 'simple', normed = True)
        df_result = df_result.append({'Method': method, 'isDisc': 'M', 'isSimple': 'S', 'isLogit': 'L', **acc, **rmse}, ignore_index=True)
        df_result.to_csv(path, index=False)

"['MESOZOIC']"
"['DEVONIAN-PERMIAN']"
"['PALEOZOIC-CRETACEOUS']"
"['GLAUCONITIC SANDSTONE', 'GAS']"
"['PALEOZOIC']"
list index out of range
list index out of range
"['MESOZOIC', 'RIFT']"
"['VOLCANICS', 'MESOZOIC']"
"['MESOZOIC']"
"['DEVONIAN-PERMIAN', 'SUB-SALT']"
"['LIMESTONE', 'DEVONIAN-PERMIAN']"
"['DEVONIAN-PERMIAN']"
list index out of range
list index out of range
"['PALEOZOIC-CRETACEOUS', 'SUB-SALT']"
"['SANDSTONE', 'PALEOZOIC-CRETACEOUS']"
"['PALEOZOIC-CRETACEOUS']"
"['GLAUCONITIC SANDSTONE', 'CRETACEOUS']"
"['PALEOZOIC', 'SUB-SALT']"
"['SANDSTONE', 'PALEOZOIC']"
"['PALEOZOIC']"
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range


d:\BAMT\bayesian\redef_HC.py:156: RuntimeWarning: overflow encountered in double_scalars
  delta_score = nrow * (mi_old - mi_new)
d:\BAMT\bayesian\redef_HC.py:156: RuntimeWarning: overflow encountered in double_scalars
  delta_score = nrow * (mi_old - mi_new)
d:\BAMT\bayesian\redef_HC.py:156: RuntimeWarning: overflow encountered in double_scalars
  delta_score = nrow * (mi_old - mi_new)
d:\BAMT\bayesian\redef_HC.py:156: RuntimeWarning: overflow encountered in double_scalars
  delta_score = nrow * (mi_old - mi_new)
d:\BAMT\bayesian\redef_HC.py:156: RuntimeWarning: overflow encountered in double_scalars
  delta_score = nrow * (mi_old - mi_new)
d:\BAMT\bayesian\redef_HC.py:156: RuntimeWarning: overflow encountered in double_scalars
  delta_score = nrow * (mi_old - mi_new)
d:\BAMT\bayesian\redef_HC.py:156: RuntimeWarning: overflow encountered in double_scalars
  delta_score = nrow * (mi_old - mi_new)
d:\BAMT\bayesian\redef_HC.py:156: RuntimeWarning: overflow encountered in double_scalars
 

list index out of range
"['EXTENSION', 'MESOZOIC']"
"['EXTENSION', 'MESOZOIC']"
"['EXTENSION', 'MESOZOIC', 'VOLCANICS']"
"['EXTENSION', 'MESOZOIC', 'VOLCANICS']"
"['EXTENSION', 'MESOZOIC', 'VOLCANICS']"
"['EXTENSION', 'MESOZOIC', 'VOLCANICS']"
"['EXTENSION', 'MESOZOIC', 'VOLCANICS']"
"['COMPRESSION', 'DEVONIAN-PERMIAN']"
"['COMPRESSION', 'DEVONIAN-PERMIAN']"
"['COMPRESSION', 'DEVONIAN-PERMIAN', 'LIMESTONE']"
"['COMPRESSION', 'DEVONIAN-PERMIAN', 'LIMESTONE']"
"['COMPRESSION', 'DEVONIAN-PERMIAN', 'LIMESTONE']"
"['COMPRESSION', 'DEVONIAN-PERMIAN', 'LIMESTONE']"
"['COMPRESSION', 'DEVONIAN-PERMIAN', 'LIMESTONE']"
list index out of range
list index out of range
"['EXTENSION', 'PALEOZOIC-CRETACEOUS']"
"['EXTENSION', 'PALEOZOIC-CRETACEOUS']"
"['EXTENSION', 'PALEOZOIC-CRETACEOUS', 'SANDSTONE']"
"['EXTENSION', 'PALEOZOIC-CRETACEOUS', 'SANDSTONE']"
"['EXTENSION', 'PALEOZOIC-CRETACEOUS', 'SANDSTONE']"
"['EXTENSION', 'PALEOZOIC-CRETACEOUS', 'SANDSTONE']"
"['EXTENSION', 'PALEOZOIC-CRETACEOUS', 'SAND

d:\BAMT\bayesian\redef_HC.py:156: RuntimeWarning: invalid value encountered in double_scalars
  delta_score = nrow * (mi_old - mi_new)
d:\BAMT\bayesian\redef_HC.py:156: RuntimeWarning: invalid value encountered in double_scalars
  delta_score = nrow * (mi_old - mi_new)
d:\BAMT\bayesian\redef_HC.py:156: RuntimeWarning: invalid value encountered in double_scalars
  delta_score = nrow * (mi_old - mi_new)
d:\BAMT\bayesian\redef_HC.py:156: RuntimeWarning: invalid value encountered in double_scalars
  delta_score = nrow * (mi_old - mi_new)
d:\BAMT\bayesian\redef_HC.py:156: RuntimeWarning: invalid value encountered in double_scalars
  delta_score = nrow * (mi_old - mi_new)
d:\BAMT\bayesian\redef_HC.py:156: RuntimeWarning: invalid value encountered in double_scalars
  delta_score = nrow * (mi_old - mi_new)
d:\BAMT\bayesian\redef_HC.py:156: RuntimeWarning: invalid value encountered in double_scalars
  delta_score = nrow * (mi_old - mi_new)
d:\BAMT\bayesian\redef_HC.py:156: RuntimeWarning: inval

"['VOLCANICS', 'OIL', 'MESOZOIC']"
"['VOLCANICS', 'MESOZOIC', 'RIFT']"
"['VOLCANICS', 'MESOZOIC', 'RIFT']"
"['LIMESTONE', 'GAS-CONDENSATE', 'DEVONIAN-PERMIAN']"
"['LIMESTONE', 'DEVONIAN-PERMIAN', 'SUB-SALT']"
"['LIMESTONE', 'DEVONIAN-PERMIAN', 'SUB-SALT']"
"['SANDSTONE', 'OIL', 'PALEOZOIC-CRETACEOUS']"
"['SANDSTONE', 'PALEOZOIC-CRETACEOUS', 'SUB-SALT']"
"['SANDSTONE', 'PALEOZOIC-CRETACEOUS', 'SUB-SALT']"
"['GLAUCONITIC SANDSTONE', 'INVERSION', 'RIFT']"
"['GLAUCONITIC SANDSTONE', 'GAS', 'INVERSION']"
"['GLAUCONITIC SANDSTONE', 'GAS', 'CRETACEOUS']"
"['GLAUCONITIC SANDSTONE', 'CRETACEOUS', 'RIFT']"
"['GLAUCONITIC SANDSTONE', 'CRETACEOUS', 'RIFT']"
"['SANDSTONE', 'OIL', 'PALEOZOIC']"
"['SANDSTONE', 'PALEOZOIC', 'SUB-SALT']"
"['SANDSTONE', 'PALEOZOIC', 'SUB-SALT']"
"['VOLCANICS', 'EXTENSION', 'MESOZOIC']"
"['OIL', 'EXTENSION', 'MESOZOIC']"
"['VOLCANICS', 'EXTENSION', 'MESOZOIC']"
"['LIMESTONE', 'COMPRESSION', 'DEVONIAN-PERMIAN']"
"['GAS-CONDENSATE', 'COMPRESSION', 'DEVONIAN-PERMIAN']"
"['L

# Logit mix

In [11]:
for method in method_list:
    bn_geo = structure_learning(geo_discrete, 'HC', geo_types, method, cont_disc = True)
    params = parameter_learning(geo_train, geo_types, bn_geo, 'mix')
    save_structure(bn_geo, path)
    skel = read_structure(path)
    save_params(params, f'{path}_param')
    params = read_params(f'{path}_param')
    geo_bn = HyBayesianNetwork(skel, params)

    acc, rmse, real_param, pred_param, indexes = calculate_acc(geo_bn, geo_test, columns, 'mix', normed = True)
    df_result = pd.read_csv(path)
    df_result = df_result.append({'Method': method, 'isDisc': 'D', 'isSimple': 'M', 'isLogit': 'L', **acc, **rmse}, ignore_index=True)

    df_result.to_csv(path, index=False)
    df_result = pd.read_csv(path)
    if method != 'K2':
        bn_geo = structure_learning(geo_coded, 'HC', geo_types, method, cont_disc = True)
        params = parameter_learning(geo_train, geo_types, bn_geo, 'mix')
        save_structure(bn_geo, path)
        skel = read_structure(path)
        save_params(params, f'{path}_param')
        params = read_params(f'{path}_param')
        geo_bn = HyBayesianNetwork(skel, params)

        acc, rmse, real_param, pred_param, indexes = calculate_acc(geo_bn, geo_test, columns, 'mix', normed = True)
        df_result = df_result.append({'Method': method, 'isDisc': 'M', 'isSimple': 'M', 'isLogit': 'L', **acc, **rmse}, ignore_index=True)
        df_result.to_csv(path, index=False)

C:\Users\AB\AppData\Roaming\Python\Python38\site-packages\sklearn\mixture\_base.py:147: ConvergenceWarning: Number of distinct clusters (6) found smaller than n_clusters (7). Possibly due to duplicate points in X.
  label = cluster.KMeans(n_clusters=self.n_components, n_init=1,
C:\Users\AB\AppData\Roaming\Python\Python38\site-packages\sklearn\mixture\_base.py:147: ConvergenceWarning: Number of distinct clusters (6) found smaller than n_clusters (7). Possibly due to duplicate points in X.
  label = cluster.KMeans(n_clusters=self.n_components, n_init=1,
C:\Users\AB\AppData\Roaming\Python\Python38\site-packages\sklearn\mixture\_base.py:147: ConvergenceWarning: Number of distinct clusters (9) found smaller than n_clusters (10). Possibly due to duplicate points in X.
  label = cluster.KMeans(n_clusters=self.n_components, n_init=1,
C:\Users\AB\AppData\Roaming\Python\Python38\site-packages\sklearn\mixture\_base.py:147: ConvergenceWarning: Number of distinct clusters (9) found smaller than n_c

"['MESOZOIC']"
"['DEVONIAN-PERMIAN']"
"['PALEOZOIC-CRETACEOUS']"
"['GLAUCONITIC SANDSTONE', 'GAS']"
"['PALEOZOIC']"
list index out of range
"['LIMESTONE', 'PALEOGENE-NEOGENE']"
list index out of range
"['SANDSTONE', 'ORDOVICIAN']"
"['MESOZOIC', 'RIFT']"
"['VOLCANICS', 'MESOZOIC']"
"['MESOZOIC']"
"['DEVONIAN-PERMIAN', 'SUB-SALT']"
"['LIMESTONE', 'DEVONIAN-PERMIAN']"
"['DEVONIAN-PERMIAN']"
list index out of range
"['DOLOMITE', 'SILURIAN']"
list index out of range
"['DOLOMITE', 'SILURIAN']"
"['PALEOZOIC-CRETACEOUS', 'SUB-SALT']"
"['SANDSTONE', 'PALEOZOIC-CRETACEOUS']"
"['PALEOZOIC-CRETACEOUS']"
"['GLAUCONITIC SANDSTONE', 'CRETACEOUS']"
"['DOLOMITE', 'CRETACEOUS']"
"['PALEOZOIC', 'SUB-SALT']"
"['SANDSTONE', 'PALEOZOIC']"
"['PALEOZOIC']"
list index out of range
list index out of range
list index out of range
list index out of range
"['DOLOMITIC LIMESTONE', 'PALEOGENE-NEOGENE']"
list index out of range
list index out of range


d:\BAMT\bayesian\redef_HC.py:156: RuntimeWarning: overflow encountered in double_scalars
  delta_score = nrow * (mi_old - mi_new)
d:\BAMT\bayesian\redef_HC.py:156: RuntimeWarning: overflow encountered in double_scalars
  delta_score = nrow * (mi_old - mi_new)
d:\BAMT\bayesian\redef_HC.py:156: RuntimeWarning: overflow encountered in double_scalars
  delta_score = nrow * (mi_old - mi_new)
d:\BAMT\bayesian\redef_HC.py:156: RuntimeWarning: overflow encountered in double_scalars
  delta_score = nrow * (mi_old - mi_new)
d:\BAMT\bayesian\redef_HC.py:156: RuntimeWarning: overflow encountered in double_scalars
  delta_score = nrow * (mi_old - mi_new)
d:\BAMT\bayesian\redef_HC.py:156: RuntimeWarning: overflow encountered in double_scalars
  delta_score = nrow * (mi_old - mi_new)
d:\BAMT\bayesian\redef_HC.py:156: RuntimeWarning: overflow encountered in double_scalars
  delta_score = nrow * (mi_old - mi_new)
d:\BAMT\bayesian\redef_HC.py:156: RuntimeWarning: overflow encountered in double_scalars
 

list index out of range
"['COMPRESSION', 'PALEOGENE-NEOGENE', 'LIMESTONE']"
"['COMPRESSION', 'PALEOGENE-NEOGENE', 'LIMESTONE']"
"['COMPRESSION', 'PALEOGENE-NEOGENE', 'LIMESTONE']"
"['COMPRESSION', 'PALEOGENE-NEOGENE', 'LIMESTONE']"
"['COMPRESSION', 'PALEOGENE-NEOGENE', 'LIMESTONE']"
"['COMPRESSION', 'ORDOVICIAN', 'SANDSTONE']"
"['COMPRESSION', 'ORDOVICIAN', 'SANDSTONE']"
"['COMPRESSION', 'ORDOVICIAN', 'SANDSTONE']"
"['COMPRESSION', 'ORDOVICIAN', 'SANDSTONE']"
"['COMPRESSION', 'ORDOVICIAN', 'SANDSTONE']"
"['EXTENSION', 'MESOZOIC']"
"['EXTENSION', 'MESOZOIC']"
"['EXTENSION', 'MESOZOIC', 'VOLCANICS']"
"['EXTENSION', 'MESOZOIC', 'VOLCANICS']"
"['EXTENSION', 'MESOZOIC', 'VOLCANICS']"
"['EXTENSION', 'MESOZOIC', 'VOLCANICS']"
"['EXTENSION', 'MESOZOIC', 'VOLCANICS']"
"['COMPRESSION', 'DEVONIAN-PERMIAN']"
"['COMPRESSION', 'DEVONIAN-PERMIAN']"
"['COMPRESSION', 'DEVONIAN-PERMIAN', 'LIMESTONE']"
"['COMPRESSION', 'DEVONIAN-PERMIAN', 'LIMESTONE']"
"['COMPRESSION', 'DEVONIAN-PERMIAN', 'LIMESTONE']"
"

d:\BAMT\bayesian\redef_HC.py:156: RuntimeWarning: invalid value encountered in double_scalars
  delta_score = nrow * (mi_old - mi_new)
d:\BAMT\bayesian\redef_HC.py:156: RuntimeWarning: invalid value encountered in double_scalars
  delta_score = nrow * (mi_old - mi_new)
d:\BAMT\bayesian\redef_HC.py:156: RuntimeWarning: invalid value encountered in double_scalars
  delta_score = nrow * (mi_old - mi_new)
d:\BAMT\bayesian\redef_HC.py:156: RuntimeWarning: invalid value encountered in double_scalars
  delta_score = nrow * (mi_old - mi_new)
d:\BAMT\bayesian\redef_HC.py:156: RuntimeWarning: invalid value encountered in double_scalars
  delta_score = nrow * (mi_old - mi_new)
d:\BAMT\bayesian\redef_HC.py:156: RuntimeWarning: invalid value encountered in double_scalars
  delta_score = nrow * (mi_old - mi_new)
d:\BAMT\bayesian\redef_HC.py:156: RuntimeWarning: invalid value encountered in double_scalars
  delta_score = nrow * (mi_old - mi_new)
d:\BAMT\bayesian\redef_HC.py:156: RuntimeWarning: inval

...

"['LIMESTONE', 'COMPRESSION', 'PALEOGENE-NEOGENE']"
"['OIL', 'COMPRESSION', 'PALEOGENE-NEOGENE']"
"['LIMESTONE', 'COMPRESSION', 'PALEOGENE-NEOGENE']"
"['SANDSTONE', 'COMPRESSION', 'ORDOVICIAN']"
"['SANDSTONE', 'COMPRESSION', 'ORDOVICIAN']"
"['VOLCANICS', 'EXTENSION', 'MESOZOIC']"
"['OIL', 'EXTENSION', 'MESOZOIC']"
"['VOLCANICS', 'EXTENSION', 'MESOZOIC']"
"['GAS', 'COMPRESSION', 'CARBONIFEROUS-PERMIAN']"
"['LIMESTONE', 'COMPRESSION', 'DEVONIAN-PERMIAN']"
"['GAS-CONDENSATE', 'COMPRESSION', 'DEVONIAN-PERMIAN']"
"['LIMESTONE', 'COMPRESSION', 'DEVONIAN-PERMIAN']"
"['DOLOMITE', 'COMPRESSION', 'SILURIAN']"
"['GAS', 'COMPRESSION', 'SILURIAN']"
"['DOLOMITE', 'COMPRESSION', 'SILURIAN']"
"['DOLOMITE', 'COMPRESSION', 'SILURIAN']"
"['GAS', 'COMPRESSION', 'SILURIAN']"
"['DOLOMITE', 'COMPRESSION', 'SILURIAN']"
"['SANDSTONE', 'EXTENSION', 'PALEOZOIC-CRETACEOUS']"
"['OIL', 'EXTENSION', 'PALEOZOIC-CRETACEOUS']"
"['SANDSTONE', 'EXTENSION', 'PALEOZOIC-CRETACEOUS']"
"['RIFT', 'GLAUCONITIC SANDSTONE', 'INVE

C:\Users\AB\AppData\Roaming\Python\Python38\site-packages\sklearn\mixture\_base.py:147: ConvergenceWarning: Number of distinct clusters (8) found smaller than n_clusters (9). Possibly due to duplicate points in X.
  label = cluster.KMeans(n_clusters=self.n_components, n_init=1,
C:\Users\AB\AppData\Roaming\Python\Python38\site-packages\sklearn\mixture\_base.py:147: ConvergenceWarning: Number of distinct clusters (8) found smaller than n_clusters (9). Possibly due to duplicate points in X.
  label = cluster.KMeans(n_clusters=self.n_components, n_init=1,
C:\Users\AB\AppData\Roaming\Python\Python38\site-packages\sklearn\mixture\_base.py:147: ConvergenceWarning: Number of distinct clusters (5) found smaller than n_clusters (6). Possibly due to duplicate points in X.
  label = cluster.KMeans(n_clusters=self.n_components, n_init=1,
C:\Users\AB\AppData\Roaming\Python\Python38\site-packages\sklearn\mixture\_base.py:147: ConvergenceWarning: Number of distinct clusters (5) found smaller than n_cl

"['LIMESTONE', 'PALEOGENE-NEOGENE', 'FORELAND']"
"['LIMESTONE', 'PALEOGENE-NEOGENE', 'FORELAND']"
"['LIMESTONE', 'PALEOGENE-NEOGENE', 'FORELAND']"
"['SANDSTONE', 'ORDOVICIAN', 'INTRACRATONIC']"
"['SANDSTONE', 'ORDOVICIAN', 'INTRACRATONIC']"
"['SANDSTONE', 'ORDOVICIAN', 'INTRACRATONIC']"
"['VOLCANICS', 'MESOZOIC', 'RIFT']"
"['VOLCANICS', 'MESOZOIC', 'RIFT']"
"['VOLCANICS', 'MESOZOIC', 'RIFT']"
"['LIMESTONE', 'DEVONIAN-PERMIAN', 'SUB-SALT']"
"['LIMESTONE', 'DEVONIAN-PERMIAN', 'SUB-SALT']"
"['LIMESTONE', 'DEVONIAN-PERMIAN', 'SUB-SALT']"
"['DOLOMITE', 'SILURIAN', 'FORELAND']"
"['DOLOMITE', 'SILURIAN', 'FORELAND']"
"['DOLOMITE', 'SILURIAN', 'FORELAND']"
"['DOLOMITE', 'SILURIAN', 'THRUST']"
"['DOLOMITE', 'SILURIAN', 'THRUST']"
"['DOLOMITE', 'SILURIAN', 'THRUST']"
"['SANDSTONE', 'PALEOZOIC-CRETACEOUS', 'SUB-SALT']"
"['SANDSTONE', 'PALEOZOIC-CRETACEOUS', 'SUB-SALT']"
"['SANDSTONE', 'PALEOZOIC-CRETACEOUS', 'SUB-SALT']"
"['GLAUCONITIC SANDSTONE', 'INVERSION', 'RIFT']"
"['GLAUCONITIC SANDSTONE', 

C:\Users\AB\AppData\Roaming\Python\Python38\site-packages\sklearn\mixture\_base.py:147: ConvergenceWarning: Number of distinct clusters (8) found smaller than n_clusters (9). Possibly due to duplicate points in X.
  label = cluster.KMeans(n_clusters=self.n_components, n_init=1,
C:\Users\AB\AppData\Roaming\Python\Python38\site-packages\sklearn\mixture\_base.py:147: ConvergenceWarning: Number of distinct clusters (8) found smaller than n_clusters (9). Possibly due to duplicate points in X.
  label = cluster.KMeans(n_clusters=self.n_components, n_init=1,
C:\Users\AB\AppData\Roaming\Python\Python38\site-packages\sklearn\mixture\_base.py:147: ConvergenceWarning: Number of distinct clusters (6) found smaller than n_clusters (7). Possibly due to duplicate points in X.
  label = cluster.KMeans(n_clusters=self.n_components, n_init=1,
C:\Users\AB\AppData\Roaming\Python\Python38\site-packages\sklearn\mixture\_base.py:147: ConvergenceWarning: Number of distinct clusters (6) found smaller than n_cl

"['LIMESTONE', 'COMPRESSION', 'PALEOGENE-NEOGENE']"
"['LIMESTONE', 'COMPRESSION', 'PALEOGENE-NEOGENE']"
"['OIL', 'LIMESTONE', 'PALEOGENE-NEOGENE']"
"['OIL', 'LIMESTONE', 'PALEOGENE-NEOGENE']"
"['FORELAND', 'LIMESTONE', 'PALEOGENE-NEOGENE']"
"['SANDSTONE', 'COMPRESSION', 'ORDOVICIAN']"
"['SANDSTONE', 'COMPRESSION', 'ORDOVICIAN']"
"['GAS', 'SANDSTONE', 'ORDOVICIAN']"
"['GAS', 'SANDSTONE', 'ORDOVICIAN']"
"['INTRACRATONIC', 'SANDSTONE', 'ORDOVICIAN']"
"['VOLCANICS', 'EXTENSION', 'MESOZOIC']"
"['VOLCANICS', 'EXTENSION', 'MESOZOIC']"
"['OIL', 'VOLCANICS', 'MESOZOIC']"
"['OIL', 'VOLCANICS', 'MESOZOIC']"
"['RIFT', 'VOLCANICS', 'MESOZOIC']"
"['GAS', 'LIMESTONE', 'CARBONIFEROUS-PERMIAN']"
"['GAS', 'LIMESTONE', 'CARBONIFEROUS-PERMIAN']"
"['LIMESTONE', 'COMPRESSION', 'DEVONIAN-PERMIAN']"
"['LIMESTONE', 'COMPRESSION', 'DEVONIAN-PERMIAN']"
"['GAS-CONDENSATE', 'LIMESTONE', 'DEVONIAN-PERMIAN']"
"['GAS-CONDENSATE', 'LIMESTONE', 'DEVONIAN-PERMIAN']"
"['SUB-SALT', 'LIMESTONE', 'DEVONIAN-PERMIAN']"
"['DOL

# Unlogit simple

In [12]:
for method in method_list:
    bn_geo = structure_learning(geo_discrete, 'HC', geo_types, method, cont_disc = False)
    params = parameter_learning(geo_train, geo_types, bn_geo, 'simple')
    save_structure(bn_geo, path)
    skel = read_structure(path)
    save_params(params, f'{path}_param')
    params = read_params(f'{path}_param')
    geo_bn = HyBayesianNetwork(skel, params)

    acc, rmse, real_param, pred_param, indexes = calculate_acc(geo_bn, geo_test, columns, 'simple', normed = True)
    df_result = pd.read_csv(path)
    df_result = df_result.append({'Method': method, 'isDisc': 'D', 'isSimple': 'S', 'isLogit': 'Unl', **acc, **rmse}, ignore_index=True)
    df_result.to_csv(path, index=False)
    df_result = pd.read_csv(path)


    if method != 'K2':
        bn_geo = structure_learning(geo_coded, 'HC', geo_types, method, cont_disc = False)
        params = parameter_learning(geo_train, geo_types, bn_geo, 'simple')
        save_structure(bn_geo, path)
        skel = read_structure(path)
        save_params(params, f'{path}_param')
        params = read_params(f'{path}_param')
        geo_bn = HyBayesianNetwork(skel, params)

        acc, rmse, real_param, pred_param, indexes = calculate_acc(geo_bn, geo_test, columns, 'simple', normed = True)
        df_result = df_result.append({'Method': method, 'isDisc': 'M', 'isSimple': 'S', 'isLogit': 'Unl', **acc, **rmse}, ignore_index=True)
        df_result.to_csv(path, index=False)

"['MESOZOIC']"
"['DEVONIAN-PERMIAN']"
"['PALEOZOIC-CRETACEOUS']"
"['GLAUCONITIC SANDSTONE', 'GAS']"
"['PALEOZOIC']"
"['OIL', 'MESOZOIC', 'RIFT']"
"['MESOZOIC', 'RIFT']"
"['MESOZOIC', 'VOLCANICS']"
"['MESOZOIC']"
"['GAS-CONDENSATE', 'DEVONIAN-PERMIAN', 'SUB-SALT']"
"['DEVONIAN-PERMIAN', 'SUB-SALT']"
"['DEVONIAN-PERMIAN', 'LIMESTONE']"
"['DEVONIAN-PERMIAN']"
"['OIL', 'PALEOZOIC-CRETACEOUS', 'SUB-SALT']"
"['PALEOZOIC-CRETACEOUS', 'SUB-SALT']"
"['PALEOZOIC-CRETACEOUS', 'SANDSTONE']"
"['PALEOZOIC-CRETACEOUS']"
"['CRETACEOUS', 'GLAUCONITIC SANDSTONE']"
"['GAS', 'RIFT', 'GLAUCONITIC SANDSTONE']"
"['OIL', 'PALEOZOIC', 'SUB-SALT']"
"['PALEOZOIC', 'SUB-SALT']"
"['PALEOZOIC', 'SANDSTONE']"
"['PALEOZOIC']"
"['OIL', 'MESOZOIC', 'RIFT']"
"['MESOZOIC', 'VOLCANICS', 'EXTENSION']"
"['MESOZOIC', 'VOLCANICS', 'EXTENSION']"
"['MESOZOIC', 'VOLCANICS', 'EXTENSION']"
"['MESOZOIC', 'VOLCANICS', 'EXTENSION']"
"['MESOZOIC', 'VOLCANICS', 'EXTENSION']"
"['GAS-CONDENSATE', 'DEVONIAN-PERMIAN', 'SUB-SALT']"
"['DEVON

d:\BAMT\bayesian\redef_HC.py:156: RuntimeWarning: invalid value encountered in double_scalars
  delta_score = nrow * (mi_old - mi_new)
d:\BAMT\bayesian\redef_HC.py:156: RuntimeWarning: invalid value encountered in double_scalars
  delta_score = nrow * (mi_old - mi_new)
d:\BAMT\bayesian\redef_HC.py:156: RuntimeWarning: invalid value encountered in double_scalars
  delta_score = nrow * (mi_old - mi_new)
d:\BAMT\bayesian\redef_HC.py:156: RuntimeWarning: invalid value encountered in double_scalars
  delta_score = nrow * (mi_old - mi_new)
d:\BAMT\bayesian\redef_HC.py:156: RuntimeWarning: invalid value encountered in double_scalars
  delta_score = nrow * (mi_old - mi_new)
d:\BAMT\bayesian\redef_HC.py:156: RuntimeWarning: invalid value encountered in double_scalars
  delta_score = nrow * (mi_old - mi_new)
d:\BAMT\bayesian\redef_HC.py:156: RuntimeWarning: invalid value encountered in double_scalars
  delta_score = nrow * (mi_old - mi_new)
d:\BAMT\bayesian\redef_HC.py:156: RuntimeWarning: inval

"['OIL', 'MESOZOIC']"
"['GAS-CONDENSATE', 'DEVONIAN-PERMIAN']"
"['OIL', 'PALEOZOIC-CRETACEOUS']"
"['GAS', 'GLAUCONITIC SANDSTONE']"
"['OIL', 'PALEOZOIC']"
"['RIFT', 'VOLCANICS', 'MESOZOIC']"
"['RIFT', 'VOLCANICS', 'MESOZOIC']"
"['OIL', 'VOLCANICS', 'MESOZOIC']"
"['RIFT', 'VOLCANICS', 'MESOZOIC']"
"['RIFT', 'VOLCANICS', 'MESOZOIC']"
"['SUB-SALT', 'LIMESTONE', 'DEVONIAN-PERMIAN']"
"['SUB-SALT', 'LIMESTONE', 'DEVONIAN-PERMIAN']"
"['GAS-CONDENSATE', 'LIMESTONE', 'DEVONIAN-PERMIAN']"
"['SUB-SALT', 'LIMESTONE', 'DEVONIAN-PERMIAN']"
"['SUB-SALT', 'LIMESTONE', 'DEVONIAN-PERMIAN']"
"['SUB-SALT', 'SANDSTONE', 'PALEOZOIC-CRETACEOUS']"
"['SUB-SALT', 'SANDSTONE', 'PALEOZOIC-CRETACEOUS']"
"['OIL', 'SANDSTONE', 'PALEOZOIC-CRETACEOUS']"
"['SUB-SALT', 'SANDSTONE', 'PALEOZOIC-CRETACEOUS']"
"['SUB-SALT', 'SANDSTONE', 'PALEOZOIC-CRETACEOUS']"
"['RIFT', 'GLAUCONITIC SANDSTONE', 'INVERSION']"
"['GAS', 'GLAUCONITIC SANDSTONE', 'INVERSION']"
"['RIFT', 'GLAUCONITIC SANDSTONE', 'CRETACEOUS']"
"['RIFT', 'GLAUCON

# Unlogit mix

In [13]:
for method in method_list:
    bn_geo = structure_learning(geo_discrete, 'HC', geo_types, method, cont_disc = False)
    params = parameter_learning(geo_train, geo_types, bn_geo, 'mix')
    save_structure(bn_geo, path)
    skel = read_structure(path)
    save_params(params, f'{path}_param')
    params = read_params(f'{path}_param')
    geo_bn = HyBayesianNetwork(skel, params)

    acc, rmse, real_param, pred_param, indexes = calculate_acc(geo_bn, geo_test, columns, 'mix', normed = True)
    df_result = pd.read_csv(path)
    df_result = df_result.append({'Method': method, 'isDisc': 'D', 'isSimple': 'M', 'isLogit': 'Unl', **acc, **rmse}, ignore_index=True)
    df_result.to_csv(path, index=False)
    df_result = pd.read_csv(path)
    if method != 'K2':
        bn_geo = structure_learning(geo_coded, 'HC', geo_types, method, cont_disc = False)
        params = parameter_learning(geo_train, geo_types, bn_geo, 'mix')
        save_structure(bn_geo, path)
        skel = read_structure(path)
        save_params(params, f'{path}_param')
        params = read_params(f'{path}_param')
        geo_bn = HyBayesianNetwork(skel, params)

        acc, rmse, real_param, pred_param, indexes = calculate_acc(geo_bn, geo_test, columns, 'mix', normed = True)
        df_result = df_result.append({'Method': method, 'isDisc': 'M', 'isSimple': 'M', 'isLogit': 'Unl', **acc, **rmse}, ignore_index=True)
        df_result.to_csv(path, index=False)

C:\Users\AB\AppData\Roaming\Python\Python38\site-packages\sklearn\mixture\_base.py:147: ConvergenceWarning: Number of distinct clusters (6) found smaller than n_clusters (7). Possibly due to duplicate points in X.
  label = cluster.KMeans(n_clusters=self.n_components, n_init=1,
C:\Users\AB\AppData\Roaming\Python\Python38\site-packages\sklearn\mixture\_base.py:147: ConvergenceWarning: Number of distinct clusters (6) found smaller than n_clusters (7). Possibly due to duplicate points in X.
  label = cluster.KMeans(n_clusters=self.n_components, n_init=1,
C:\Users\AB\AppData\Roaming\Python\Python38\site-packages\sklearn\mixture\_base.py:147: ConvergenceWarning: Number of distinct clusters (9) found smaller than n_clusters (10). Possibly due to duplicate points in X.
  label = cluster.KMeans(n_clusters=self.n_components, n_init=1,
C:\Users\AB\AppData\Roaming\Python\Python38\site-packages\sklearn\mixture\_base.py:147: ConvergenceWarning: Number of distinct clusters (9) found smaller than n_c

"['MESOZOIC']"
"['DEVONIAN-PERMIAN']"
"['PALEOZOIC-CRETACEOUS']"
"['GLAUCONITIC SANDSTONE', 'GAS']"
"['PALEOZOIC']"


C:\Users\AB\AppData\Roaming\Python\Python38\site-packages\sklearn\mixture\_base.py:147: ConvergenceWarning: Number of distinct clusters (5) found smaller than n_clusters (6). Possibly due to duplicate points in X.
  label = cluster.KMeans(n_clusters=self.n_components, n_init=1,
C:\Users\AB\AppData\Roaming\Python\Python38\site-packages\sklearn\mixture\_base.py:147: ConvergenceWarning: Number of distinct clusters (5) found smaller than n_clusters (6). Possibly due to duplicate points in X.
  label = cluster.KMeans(n_clusters=self.n_components, n_init=1,
C:\Users\AB\AppData\Roaming\Python\Python38\site-packages\sklearn\mixture\_base.py:147: ConvergenceWarning: Number of distinct clusters (9) found smaller than n_clusters (10). Possibly due to duplicate points in X.
  label = cluster.KMeans(n_clusters=self.n_components, n_init=1,
C:\Users\AB\AppData\Roaming\Python\Python38\site-packages\sklearn\mixture\_base.py:147: ConvergenceWarning: Number of distinct clusters (9) found smaller than n_c

"['PALEOGENE-NEOGENE', 'FORELAND']"
"['PALEOGENE-NEOGENE', 'LIMESTONE']"
"['ORDOVICIAN', 'INTRACRATONIC']"
"['ORDOVICIAN', 'SANDSTONE']"
"['OIL', 'MESOZOIC', 'RIFT']"
"['MESOZOIC', 'RIFT']"
"['MESOZOIC', 'VOLCANICS']"
"['OIL', 'RIFT', 'VOLCANICS']"
"['MESOZOIC']"
"['GAS', 'SUB-SALT', 'LIMESTONE']"
"['GAS-CONDENSATE', 'DEVONIAN-PERMIAN', 'SUB-SALT']"
"['DEVONIAN-PERMIAN', 'SUB-SALT']"
"['DEVONIAN-PERMIAN', 'LIMESTONE']"
"['GAS-CONDENSATE', 'SUB-SALT', 'LIMESTONE']"
"['DEVONIAN-PERMIAN']"
"['SILURIAN', 'FORELAND']"
"['SILURIAN', 'DOLOMITE']"
"['SILURIAN', 'THRUST']"
"['SILURIAN', 'DOLOMITE']"
"['OIL', 'PALEOZOIC-CRETACEOUS', 'SUB-SALT']"
"['PALEOZOIC-CRETACEOUS', 'SUB-SALT']"
"['PALEOZOIC-CRETACEOUS', 'SANDSTONE']"
"['PALEOZOIC-CRETACEOUS']"
"['CRETACEOUS', 'GLAUCONITIC SANDSTONE']"
"['GAS', 'RIFT', 'GLAUCONITIC SANDSTONE']"
"['CRETACEOUS', 'DOLOMITE']"
"['OIL', 'PALEOZOIC', 'SUB-SALT']"
"['PALEOZOIC', 'SUB-SALT']"
"['PALEOZOIC', 'SANDSTONE']"
"['PALEOZOIC']"
"['JURASSIC', 'SUB-THRUST']"

C:\Users\AB\AppData\Roaming\Python\Python38\site-packages\sklearn\mixture\_base.py:147: ConvergenceWarning: Number of distinct clusters (8) found smaller than n_clusters (9). Possibly due to duplicate points in X.
  label = cluster.KMeans(n_clusters=self.n_components, n_init=1,
C:\Users\AB\AppData\Roaming\Python\Python38\site-packages\sklearn\mixture\_base.py:147: ConvergenceWarning: Number of distinct clusters (8) found smaller than n_clusters (9). Possibly due to duplicate points in X.
  label = cluster.KMeans(n_clusters=self.n_components, n_init=1,
C:\Users\AB\AppData\Roaming\Python\Python38\site-packages\sklearn\mixture\_base.py:147: ConvergenceWarning: Number of distinct clusters (7) found smaller than n_clusters (8). Possibly due to duplicate points in X.
  label = cluster.KMeans(n_clusters=self.n_components, n_init=1,
C:\Users\AB\AppData\Roaming\Python\Python38\site-packages\sklearn\mixture\_base.py:147: ConvergenceWarning: Number of distinct clusters (7) found smaller than n_cl

...

"['LIMESTONE', 'COMPRESSION', 'PALEOGENE-NEOGENE']"
"['OIL', 'COMPRESSION', 'PALEOGENE-NEOGENE']"
"['LIMESTONE', 'COMPRESSION', 'PALEOGENE-NEOGENE']"
"['SANDSTONE', 'COMPRESSION', 'ORDOVICIAN']"
"['SANDSTONE', 'COMPRESSION', 'ORDOVICIAN']"
"['VOLCANICS', 'EXTENSION', 'MESOZOIC']"
"['OIL', 'EXTENSION', 'MESOZOIC']"
"['VOLCANICS', 'EXTENSION', 'MESOZOIC']"
"['GAS', 'COMPRESSION', 'CARBONIFEROUS-PERMIAN']"
"['LIMESTONE', 'COMPRESSION', 'DEVONIAN-PERMIAN']"
"['GAS-CONDENSATE', 'COMPRESSION', 'DEVONIAN-PERMIAN']"
"['LIMESTONE', 'COMPRESSION', 'DEVONIAN-PERMIAN']"
"['DOLOMITE', 'COMPRESSION', 'SILURIAN']"
"['GAS', 'COMPRESSION', 'SILURIAN']"
"['DOLOMITE', 'COMPRESSION', 'SILURIAN']"
"['DOLOMITE', 'COMPRESSION', 'SILURIAN']"
"['GAS', 'COMPRESSION', 'SILURIAN']"
"['DOLOMITE', 'COMPRESSION', 'SILURIAN']"
"['SANDSTONE', 'EXTENSION', 'PALEOZOIC-CRETACEOUS']"
"['OIL', 'EXTENSION', 'PALEOZOIC-CRETACEOUS']"
"['SANDSTONE', 'EXTENSION', 'PALEOZOIC-CRETACEOUS']"
"['RIFT', 'GLAUCONITIC SANDSTONE', 'INVE

C:\Users\AB\AppData\Roaming\Python\Python38\site-packages\sklearn\mixture\_base.py:147: ConvergenceWarning: Number of distinct clusters (6) found smaller than n_clusters (7). Possibly due to duplicate points in X.
  label = cluster.KMeans(n_clusters=self.n_components, n_init=1,
C:\Users\AB\AppData\Roaming\Python\Python38\site-packages\sklearn\mixture\_base.py:147: ConvergenceWarning: Number of distinct clusters (6) found smaller than n_clusters (7). Possibly due to duplicate points in X.
  label = cluster.KMeans(n_clusters=self.n_components, n_init=1,
C:\Users\AB\AppData\Roaming\Python\Python38\site-packages\sklearn\mixture\_base.py:147: ConvergenceWarning: Number of distinct clusters (8) found smaller than n_clusters (9). Possibly due to duplicate points in X.
  label = cluster.KMeans(n_clusters=self.n_components, n_init=1,
C:\Users\AB\AppData\Roaming\Python\Python38\site-packages\sklearn\mixture\_base.py:147: ConvergenceWarning: Number of distinct clusters (8) found smaller than n_cl

"['FORELAND', 'LIMESTONE', 'PALEOGENE-NEOGENE']"
"['FORELAND', 'LIMESTONE', 'PALEOGENE-NEOGENE']"
"['FORELAND', 'LIMESTONE', 'PALEOGENE-NEOGENE']"
"['FORELAND', 'LIMESTONE', 'PALEOGENE-NEOGENE']"
"['FORELAND', 'LIMESTONE', 'PALEOGENE-NEOGENE']"
"['INTRACRATONIC', 'SANDSTONE', 'ORDOVICIAN']"
"['INTRACRATONIC', 'SANDSTONE', 'ORDOVICIAN']"
"['INTRACRATONIC', 'SANDSTONE', 'ORDOVICIAN']"
"['INTRACRATONIC', 'SANDSTONE', 'ORDOVICIAN']"
"['INTRACRATONIC', 'SANDSTONE', 'ORDOVICIAN']"
"['RIFT', 'VOLCANICS', 'MESOZOIC']"
"['RIFT', 'VOLCANICS', 'MESOZOIC']"
"['RIFT', 'VOLCANICS', 'MESOZOIC']"
"['RIFT', 'VOLCANICS', 'MESOZOIC']"
"['RIFT', 'VOLCANICS', 'MESOZOIC']"
"['SUB-SALT', 'LIMESTONE', 'DEVONIAN-PERMIAN']"
"['SUB-SALT', 'LIMESTONE', 'DEVONIAN-PERMIAN']"
"['SUB-SALT', 'LIMESTONE', 'DEVONIAN-PERMIAN']"
"['SUB-SALT', 'LIMESTONE', 'DEVONIAN-PERMIAN']"
"['SUB-SALT', 'LIMESTONE', 'DEVONIAN-PERMIAN']"
"['FORELAND', 'DOLOMITE', 'SILURIAN']"
"['FORELAND', 'DOLOMITE', 'SILURIAN']"
"['FORELAND', 'DOLOMIT

C:\Users\AB\AppData\Roaming\Python\Python38\site-packages\sklearn\mixture\_base.py:147: ConvergenceWarning: Number of distinct clusters (8) found smaller than n_clusters (9). Possibly due to duplicate points in X.
  label = cluster.KMeans(n_clusters=self.n_components, n_init=1,
C:\Users\AB\AppData\Roaming\Python\Python38\site-packages\sklearn\mixture\_base.py:147: ConvergenceWarning: Number of distinct clusters (8) found smaller than n_clusters (9). Possibly due to duplicate points in X.
  label = cluster.KMeans(n_clusters=self.n_components, n_init=1,
C:\Users\AB\AppData\Roaming\Python\Python38\site-packages\sklearn\mixture\_base.py:147: ConvergenceWarning: Number of distinct clusters (6) found smaller than n_clusters (7). Possibly due to duplicate points in X.
  label = cluster.KMeans(n_clusters=self.n_components, n_init=1,
C:\Users\AB\AppData\Roaming\Python\Python38\site-packages\sklearn\mixture\_base.py:147: ConvergenceWarning: Number of distinct clusters (6) found smaller than n_cl

"['LIMESTONE', 'COMPRESSION', 'PALEOGENE-NEOGENE']"
"['LIMESTONE', 'COMPRESSION', 'PALEOGENE-NEOGENE']"
"['OIL', 'LIMESTONE', 'PALEOGENE-NEOGENE']"
"['OIL', 'LIMESTONE', 'PALEOGENE-NEOGENE']"
"['FORELAND', 'LIMESTONE', 'PALEOGENE-NEOGENE']"
"['SANDSTONE', 'COMPRESSION', 'ORDOVICIAN']"
"['SANDSTONE', 'COMPRESSION', 'ORDOVICIAN']"
"['GAS', 'SANDSTONE', 'ORDOVICIAN']"
"['GAS', 'SANDSTONE', 'ORDOVICIAN']"
"['INTRACRATONIC', 'SANDSTONE', 'ORDOVICIAN']"
"['VOLCANICS', 'EXTENSION', 'MESOZOIC']"
"['VOLCANICS', 'EXTENSION', 'MESOZOIC']"
"['OIL', 'VOLCANICS', 'MESOZOIC']"
"['OIL', 'VOLCANICS', 'MESOZOIC']"
"['RIFT', 'VOLCANICS', 'MESOZOIC']"
"['GAS', 'LIMESTONE', 'CARBONIFEROUS-PERMIAN']"
"['GAS', 'LIMESTONE', 'CARBONIFEROUS-PERMIAN']"
"['LIMESTONE', 'COMPRESSION', 'DEVONIAN-PERMIAN']"
"['LIMESTONE', 'COMPRESSION', 'DEVONIAN-PERMIAN']"
"['GAS-CONDENSATE', 'LIMESTONE', 'DEVONIAN-PERMIAN']"
"['GAS-CONDENSATE', 'LIMESTONE', 'DEVONIAN-PERMIAN']"
"['SUB-SALT', 'LIMESTONE', 'DEVONIAN-PERMIAN']"
"['DOL